<a href="https://colab.research.google.com/github/Gonzalo5151/tensorflow2/blob/master/Colab%203%20-%20Crear%20una%20Red%20Neuronal%20Artificial%20en%20TensorFlow%202.0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/joanby/tensorflow2/blob/master/Colab%203%20-%20Crear%20una%20Red%20Neuronal%20Artificial%20en%20TensorFlow%202.0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<p align="center">
  <img src="https://storage.googleapis.com/kaggle-datasets-images/2243/3791/9384af51de8baa77f6320901f53bd26b/dataset-cover.png" />
  Imagen cortesía de: https://www.kaggle.com/
</p>

## Paso 1: Instalar las dependencias y configurar el entorno de GPU

In [ ]:
#!pip install tensorflow-gpu==2.3.0
%tensorflow_version 2.x

     |████████████████████████████████| 332.1MB 93kB/s 
     |████████████████████████████████| 419kB 54.1MB/s 
     |████████████████████████████████| 3.0MB 44.6MB/s 


## Paso 2: Importar las dependencias necesarias para el proyecto

In [1]:
import numpy as np
import datetime
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist

In [2]:
tf.__version__

'2.17.0'

## Paso 3: Pre procesado de datos



### Cargar el dataset

In [81]:
#Cargar el dataset Fashion Mnist
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

### Normalizar las imágenes

Se divide cada imagen en los conjunto de entrenamiento y  de testing entre el valor máximo de cada uno de los píxeles (255).

De este modo, cada píxel se hallará en el rango [0, 1]. Al normalizar las imágenes, nos aseguramos que nuestro modelo de RNA entrenará más rápidamente.

In [82]:
X_train = X_train / 255.0

In [83]:
X_test = X_test / 255.0

### Redimensionar el dataset

Como vamos a utilizar una red neuronal totalmente conectada, vamos a redimensionar los subconjuntos de entrenamiento y testing a formato de vector en lugar de en formato de matriz.

In [84]:
#Como cada imagen tiene 28x28 píxeles, usamos la función reshape en todo el dataset de entrenamiento para convertirlo
# en vectores de tamaño [-1 (todos los elementos), anchura * altura]
X_train = X_train.reshape(-1, 28*28)

In [85]:
X_train.shape

(60000, 784)

In [86]:
#Redimensionamos el conjunto de testing del mismo modo
X_test = X_test.reshape(-1, 28*28)

## Paso 4: Construir la Red Neuronal Artificial

### Definir el modelo

Simplemente se define un objeto de modelo Sequential.

In [120]:
model = tf.keras.models.Sequential()

### Añadir la primera capa totalmente conectada (capa Densa)

Hyper-parametros de la capa:
- número de unidades/neuronas: 128
- función de activación: ReLU
- input_shape: (784, )

In [121]:
model.add(tf.keras.layers.Dense(units=128, activation='relu', input_shape=(784, )))
#model.add(tf.keras.layers.Dense(units=64, activation='relu' ))


### Añadir una capa de Dropout

Dropout es una técnica de Regularization donde aleatoriamente se asignan a ciertas neuronas de la red el valor cero. De este modo, mientras se entrena, estas neuronas no actualizarán sus valores. Al tener cierto porcentaje de neuronas sin actualizar, el proceso de entrenamiento toma más tiempo pero por contra tenemos menos posibilidades de sufrir overfitting.

In [122]:
model.add(tf.keras.layers.Dropout(0.2))

### Añadir la segunda capa (capa de salida)

- unidades: número de clases (10 en el caso del Fashion MNIST)
- función de activación: 'softmax' (probabilidades de cada clase)

In [123]:
model.add(tf.keras.layers.Dense(units=10, activation='softmax'))

### Compilar el modelo

- Optimizer: Adam
- Loss: Sparse softmax (categorical) crossentropy

In [124]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])


In [125]:
model.summary()

Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_26 (Dense)                     │ (None, 128)                 │         100,480 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_10 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_27 (Dense)                     │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 101,770 (397.54 KB)

 Trainable params: 101,770 (397.54 KB)

 Non-trainable params: 0 (0.00 B)

### Entrenar el modelo

In [126]:
callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    start_from_epoch=0)
model.fit(X_train, y_train,  validation_data=(X_test, y_test), epochs=100, callbacks=[callback])


Epoch 1/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - loss: 1.0559 - sparse_categorical_accuracy: 0.6575 - val_loss: 0.5441 - val_sparse_categorical_accuracy: 0.8197
Epoch 2/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - loss: 0.5386 - sparse_categorical_accuracy: 0.8178 - val_loss: 0.4798 - val_sparse_categorical_accuracy: 0.8350
Epoch 3/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.4724 - sparse_categorical_accuracy: 0.8348 - val_loss: 0.4448 - val_sparse_categorical_accuracy: 0.8446
Epoch 4/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.4348 - sparse_categorical_accuracy: 0.8483 - val_loss: 0.4258 - val_sparse_categorical_accuracy: 0.8501
Epoch 5/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 0.4104 - sparse_categorical_accuracy: 0.8593 - val_loss: 0.4111 - val_sparse_categorical_accuracy: 0.8535
Epoch 6/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.3908 - sparse_categorical_accuracy: 0.8625 - val_loss: 0.3967 - val_sparse_categorical_accuracy:

### Evaluación del modelo y predicción

In [127]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.3062 - sparse_categorical_accuracy: 0.8922


In [128]:
print("Test accuracy: {}".format(test_accuracy))

Test accuracy: 0.8931000232696533


## Paso 5 : Guardar el modelo

### Guardar la arquitectura (topoligía) de la red neuronal

In [ ]:
model_json = model.to_json()
with open("fashion_model.json", "w") as json_file:
    json_file.write(model_json)

### Guardar los pesos de la red neuronal

In [ ]:
model.save_weights("fashion_model.h5")